In [6]:
import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
lista=['ETHUSDT']

suma=0.0

for symbol in lista:
    data=util.obtiene_historial(symbol,'1h')
    distancia= abs(data.Close - data.ema200)
    data['ema100']=ta.ema(data.Close, length=100)
    ##    
    data['trade'] = np.where(util.crossover_dataframe(data.ema100, data.ema200),1,np.where(util.crossover_dataframe(data.ema200, data.ema100),-1,0))
    data['stop_loss'] = np.where(data.trade==1,data.Close-distancia,np.where(data.trade==-1,data.Close+distancia,0))
    data['take_profit'] = np.where(data.trade==1,data.Close+distancia*3,np.where(data.trade==-1,data.Close-distancia*3,0))   
    ##
    bt = Backtest(data,util.backtesting_config,cash=100000,exclusive_orders=True)
    output = bt.run()
    if len(lista)==1:
        bt.plot(filename="graficos/"+symbol)
    print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)}")
    suma=suma+output['Return [%]']

suma

Symbol: ETHUSDT - Return [%]: -7.43


-7.428397459176588